In [0]:
main_path = "/content/drive/My Drive/Colab Notebooks/datasets/"#データの読み込み
import numpy as np
x_val = np.load(main_path + "x_val.npy")
y_val = np.load(main_path + "y_val.npy")
x_train= np.load(main_path + "x_train.npy")
y_train= np.load(main_path + "y_train.npy")
x_test= np.load(main_path + "x_test.npy")
y_test= np.load(main_path + "y_test.npy")

In [0]:
x_train=x_train/255
x_val=x_val/255
x_test=x_test/255
x_train=x_train.reshape(8980,224,224,1)
x_val=x_val.reshape(1448,224,224,1)
x_test=x_test.reshape(2458,224,224,1)

def shuffle(x,y):
  np.random.seed(1)
  np.random.shuffle(x)
  np.random.seed(1)
  np.random.shuffle(y)

shuffle(x_train,y_train)
shuffle(x_val,y_val)
shuffle(x_test,y_test)
import keras
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(224,224,1)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))      
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
             optimizer="rmsprop",
              metrics=['accuracy'])
es_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')
tb_cb = keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)
history = model.fit(x_train, y_train, batch_size=128, epochs=5,
                   validation_data = (x_val, y_val),verbose=1,callbacks=[es_cb, tb_cb])

Using TensorFlow backend.


Train on 8980 samples, validate on 1448 samples
Epoch 1/5
8980/8980 [==============================] - 53s 6ms/step - loss: 0.6663 - accuracy: 0.7389 - val_loss: 0.5640 - val_accuracy: 0.7762
Epoch 2/5
8980/8980 [==============================] - 49s 5ms/step - loss: 0.4012 - accuracy: 0.8297 - val_loss: 0.4952 - val_accuracy: 0.7762
Epoch 3/5
8980/8980 [==============================] - 49s 5ms/step - loss: 0.2998 - accuracy: 0.8867 - val_loss: 0.4067 - val_accuracy: 0.8391
Epoch 4/5
8980/8980 [==============================] - 49s 6ms/step - loss: 0.1903 - accuracy: 0.9322 - val_loss: 0.5079 - val_accuracy: 0.8225


In [0]:
model.save(main_path+"kadai2.h5")

In [2]:
#作成済みのモデルを呼び出す
import keras
model_new=keras.models.load_model("/content/drive/My Drive/Colab Notebooks/datasets/kadai2.h5")
model_new.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      320       
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 222, 222, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 111, 111, 64)     

In [0]:
test_loss,test_acc=model_new.evaluate(x_test,y_test)

2458/2458 [==============================] - 5s 2ms/step


In [0]:
test_acc

0.8852725625038147

In [4]:
import numpy as np
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
 
predict_classes = model_new.predict_classes(x_test)
true_classes = y_test 
def print_cmx(y_true, y_pred):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    plt.figure(figsize = (10,10))
    sn.heatmap(df_cmx, annot=True, fmt='g' ,square = True)
    plt.show()
 
print_cmx(true_classes, predict_classes)
cm = confusion_matrix(true_classes,predict_classes)

NameError: ignored

In [0]:
import gc, keras, os
import numpy as np
from keras.models import load_model
from keras.datasets import mnist
from PIL import Image, ImageDraw
pred = model_new.predict_classes(x_test)# Write error images
for i, (p, y) in enumerate(zip(pred, y_test)):
    if p != y:
        image = x_test[i]*255
        img = Image.fromarray(image.reshape((image.shape[0], image.shape[1])).astype(np.uint8))
        print(img)
        img.save(main_path+"result_img/"+"{0}-c{1}-p{2}.png".format(i, y, p)) # c=correct, p=predi